# Experiment 088: Ens Model with Yield Renormalization

**Strategy**: Implement techniques from the "Ens Model" kernel that we haven't tried:
1. Correlation-based feature filtering (threshold=0.90)
2. Yield renormalization (clip to [0,1], normalize so sum ≤ 1)
3. Different ensemble weights for single vs full data
4. Combine ALL feature sources: spange + acs_pca + drfps + fragprints

**Hypothesis**: These domain-specific constraints might change the CV-LB relationship by:
- Reducing overfitting through better feature selection
- Enforcing physical constraints (yields must sum to ≤ 1)
- Adapting to different characteristics of single vs mixture data

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
import catboost as cb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load ALL feature sources
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)
FRAGPRINTS_DF = pd.read_csv(f'{DATA_PATH}/fragprints_lookup.csv', index_col=0)

print(f'Spange: {SPANGE_DF.shape}')
print(f'DRFP: {DRFP_DF.shape}')
print(f'ACS PCA: {ACS_PCA_DF.shape}')
print(f'Fragprints: {FRAGPRINTS_DF.shape}')

Spange: (26, 13)
DRFP: (24, 2048)
ACS PCA: (24, 5)
Fragprints: (24, 2133)


In [4]:
# Correlation-based feature filtering (from Ens Model kernel)
def feature_priority(name: str) -> int:
    """Assign priority score to feature name based on prefix."""
    if name.startswith("spange_"):
        return 5
    if name.startswith("acs_"):
        return 4
    if name.startswith("drfps_"):
        return 3
    if name.startswith("frag_"):
        return 2
    return 0

def filter_correlated_features(df: pd.DataFrame, threshold: float = 0.90):
    """Drop columns that are highly correlated, keeping higher priority features."""
    numeric_df = df.select_dtypes(include=[np.number])
    
    if numeric_df.shape[1] == 0:
        return df, []
    
    # Drop constant columns first
    std = numeric_df.std(axis=0)
    constant_cols = std[std == 0].index.tolist()
    if constant_cols:
        numeric_df = numeric_df.drop(columns=constant_cols)
    
    # Correlation matrix
    corr = numeric_df.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool)).fillna(0.0)
    
    cols = upper.columns.tolist()
    to_drop = set()
    
    # Find high correlation pairs
    for i, col_i in enumerate(cols):
        for j in range(i + 1, len(cols)):
            col_j = cols[j]
            cval = upper.iloc[i, j]
            if cval > threshold:
                if col_i in to_drop or col_j in to_drop:
                    continue
                p_i = feature_priority(col_i)
                p_j = feature_priority(col_j)
                if p_i > p_j:
                    to_drop.add(col_j)
                elif p_j > p_i:
                    to_drop.add(col_i)
                else:
                    # Same priority - drop the one that appears later
                    to_drop.add(col_j)
    
    all_to_drop = list(set(constant_cols).union(to_drop))
    df_filtered = df.drop(columns=all_to_drop, errors="ignore")
    
    return df_filtered, all_to_drop

print('Feature filtering functions defined')

Feature filtering functions defined


In [5]:
# Build combined solvent feature table with correlation filtering
def build_combined_features():
    """Build combined solvent feature table from all sources."""
    # Add prefixes to distinguish sources
    spange = SPANGE_DF.copy()
    spange.columns = [f'spange_{c}' for c in spange.columns]
    
    drfp = DRFP_DF.copy()
    drfp.columns = [f'drfps_{c}' for c in drfp.columns]
    
    acs = ACS_PCA_DF.copy()
    acs.columns = [f'acs_{c}' for c in acs.columns]
    
    frag = FRAGPRINTS_DF.copy()
    frag.columns = [f'frag_{c}' for c in frag.columns]
    
    # Combine all features
    combined = spange.join(acs, how='outer').join(drfp, how='outer').join(frag, how='outer')
    combined = combined.fillna(0)
    
    print(f'Combined features before filtering: {combined.shape}')
    
    # Apply correlation filtering
    combined_filtered, dropped = filter_correlated_features(combined, threshold=0.90)
    
    print(f'Combined features after filtering: {combined_filtered.shape}')
    print(f'Dropped {len(dropped)} columns')
    
    return combined_filtered

COMBINED_FEATURES = build_combined_features()
print(f'\nFinal feature table: {COMBINED_FEATURES.shape}')

Combined features before filtering: (26, 4199)


Combined features after filtering: (26, 85)
Dropped 4114 columns

Final feature table: (26, 85)


In [6]:
# Yield renormalization function
def renormalize_predictions(preds):
    """Clip predictions to [0, inf] and normalize so sum ≤ 1."""
    # Clip to non-negative
    preds = np.clip(preds, 0, None)
    
    # Normalize if sum > 1
    totals = preds.sum(axis=1, keepdims=True)
    mask = totals > 1.0
    preds = np.where(mask, preds / totals, preds)
    
    return preds

print('Yield renormalization function defined')

Yield renormalization function defined


In [7]:
# Featurizer with combined features and engineered features
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.combined_df = COMBINED_FEATURES
        self.feats_dim = 6 + self.combined_df.shape[1]  # 6 engineered + combined

    def featurize(self, X):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        time_m = X_vals[:, 0:1]
        temp_c = X_vals[:, 1:2]
        
        # Convert to Kelvin
        temp_k = temp_c + 273.15
        
        # Engineered features (from Ens Model kernel)
        T_x_RT = temp_k * time_m  # Temperature × Residence Time
        RT_log = np.log(time_m + 1e-6)  # Log residence time
        T_inv = 1.0 / temp_k  # Inverse temperature
        RT_scaled = time_m / time_m.mean()  # Scaled residence time
        
        X_engineered = np.hstack([time_m, temp_c, T_x_RT, RT_log, T_inv, RT_scaled])
        
        if self.mixed:
            A_feats = self.combined_df.loc[X["SOLVENT A NAME"]].values
            B_feats = self.combined_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            X_solvent = A_feats * (1 - pct) + B_feats * pct
        else:
            X_solvent = self.combined_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_engineered, X_solvent])

print(f'Combined feature dimension: {CombinedFeaturizer().feats_dim}')

Combined feature dimension: 91


In [8]:
# CatBoost + XGBoost Ensemble with different weights for single vs full
class EnsModelRenorm:
    def __init__(self, data='single'):
        self.data = data
        self.mixed = (data == 'full')
        self.featurizer = CombinedFeaturizer(mixed=self.mixed)
        self.scalers = [StandardScaler() for _ in range(3)]
        self.catboost_models = []
        self.xgb_models = []
        
        # Different weights for single vs full (from Ens Model kernel)
        if self.data == 'single':
            self.cat_weight = 7 / 13  # CatBoost weight
            self.xgb_weight = 6 / 13  # XGBoost weight
        else:
            self.cat_weight = 1 / 3  # CatBoost weight
            self.xgb_weight = 2 / 3  # XGBoost weight
        
    def train_model(self, X, Y):
        X_feats = self.featurizer.featurize(X)
        Y_vals = Y.values
        
        self.catboost_models = []
        self.xgb_models = []
        
        for i in range(3):
            y_scaled = self.scalers[i].fit_transform(Y_vals[:, i:i+1]).ravel()
            
            # CatBoost
            cat_model = cb.CatBoostRegressor(
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3,
                random_seed=42,
                verbose=False
            )
            cat_model.fit(X_feats, y_scaled)
            self.catboost_models.append(cat_model)
            
            # XGBoost
            xgb_model = xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                reg_lambda=3,
                random_state=42,
                verbosity=0
            )
            xgb_model.fit(X_feats, y_scaled)
            self.xgb_models.append(xgb_model)
    
    def predict(self, X):
        X_feats = self.featurizer.featurize(X)
        preds = []
        
        for i in range(3):
            cat_pred = self.catboost_models[i].predict(X_feats)
            xgb_pred = self.xgb_models[i].predict(X_feats)
            
            # Weighted ensemble
            pred_scaled = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
            pred = self.scalers[i].inverse_transform(pred_scaled.reshape(-1, 1)).ravel()
            preds.append(pred)
        
        preds = np.stack(preds, axis=1)
        
        # Apply yield renormalization
        preds = renormalize_predictions(preds)
        
        return torch.tensor(preds)

print('EnsModelRenorm defined')

EnsModelRenorm defined


In [ ]:
# Cross-validation on single solvent data
import tqdm

X, Y = load_data("single_solvent")
print(f'Single solvent data: {len(X)} samples, {len(X["SOLVENT NAME"].unique())} solvents')

all_mse = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_out_splits(X, Y))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = EnsModelRenorm(data='single')
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse.append(mse)

single_mse = np.mean(all_mse)
print(f'\nSingle Solvent MSE: {single_mse:.6f}')

In [ ]:
# Cross-validation on full data
X_full, Y_full = load_data("full")
print(f'Full data: {len(X_full)} samples')

all_mse_full = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = EnsModelRenorm(data='full')
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse_full.append(mse)

full_mse = np.mean(all_mse_full)
print(f'\nFull Data MSE: {full_mse:.6f}')

In [ ]:
# Calculate overall CV score
n_single = 656
n_full = 1227
overall_mse = (single_mse * n_single + full_mse * n_full) / (n_single + n_full)

print(f'\n=== Ens Model with Renormalization Results ===')
print(f'Single Solvent MSE: {single_mse:.6f}')
print(f'Full Data MSE: {full_mse:.6f}')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest baseline (GP+MLP+LGBM): 0.008298')
print(f'Difference: {(overall_mse - 0.008298) / 0.008298 * 100:.2f}%')

In [ ]:
# Compare with and without renormalization
print('\n=== Effect of Yield Renormalization ===')

# Test without renormalization
class EnsModelNoRenorm:
    def __init__(self, data='single'):
        self.data = data
        self.mixed = (data == 'full')
        self.featurizer = CombinedFeaturizer(mixed=self.mixed)
        self.scalers = [StandardScaler() for _ in range(3)]
        self.catboost_models = []
        self.xgb_models = []
        
        if self.data == 'single':
            self.cat_weight = 7 / 13
            self.xgb_weight = 6 / 13
        else:
            self.cat_weight = 1 / 3
            self.xgb_weight = 2 / 3
        
    def train_model(self, X, Y):
        X_feats = self.featurizer.featurize(X)
        Y_vals = Y.values
        
        self.catboost_models = []
        self.xgb_models = []
        
        for i in range(3):
            y_scaled = self.scalers[i].fit_transform(Y_vals[:, i:i+1]).ravel()
            
            cat_model = cb.CatBoostRegressor(
                iterations=500, learning_rate=0.05, depth=6,
                l2_leaf_reg=3, random_seed=42, verbose=False
            )
            cat_model.fit(X_feats, y_scaled)
            self.catboost_models.append(cat_model)
            
            xgb_model = xgb.XGBRegressor(
                n_estimators=500, learning_rate=0.05, max_depth=6,
                reg_lambda=3, random_state=42, verbosity=0
            )
            xgb_model.fit(X_feats, y_scaled)
            self.xgb_models.append(xgb_model)
    
    def predict(self, X):
        X_feats = self.featurizer.featurize(X)
        preds = []
        
        for i in range(3):
            cat_pred = self.catboost_models[i].predict(X_feats)
            xgb_pred = self.xgb_models[i].predict(X_feats)
            pred_scaled = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
            pred = self.scalers[i].inverse_transform(pred_scaled.reshape(-1, 1)).ravel()
            preds.append(pred)
        
        return torch.tensor(np.stack(preds, axis=1))

# Quick test on single solvent
X, Y = load_data("single_solvent")
all_mse_no_renorm = []
for fold_idx, split in enumerate(generate_leave_one_out_splits(X, Y)):
    (train_X, train_Y), (test_X, test_Y) = split
    model = EnsModelNoRenorm(data='single')
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mse = np.mean((preds - test_Y.values) ** 2)
    all_mse_no_renorm.append(mse)

print(f'Single Solvent MSE without renorm: {np.mean(all_mse_no_renorm):.6f}')
print(f'Single Solvent MSE with renorm: {single_mse:.6f}')
print(f'Effect of renormalization: {(single_mse - np.mean(all_mse_no_renorm)) / np.mean(all_mse_no_renorm) * 100:.2f}%')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = EnsModelRenorm(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = EnsModelRenorm(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################